In [1]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import time
import locale
import pandas as pd
import cchardet
import threading
import random

locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"}

In [5]:
options = webdriver.ChromeOptions()
options.add_argument('headless')
capa = DesiredCapabilities.CHROME
capa["pageLoadStrategy"] = "none"
driver = webdriver.Chrome(options=options, desired_capabilities=capa)
driver.set_window_size(1440,900)
requests_session = requests.Session()
link = '/search/title/?moviemeter=0,&sort=num_votes,desc&count=250'
# endgame
# link = '/search/title/?moviemeter=,1'
details = ['title', 'runtime', 'certificate', 'rating', 'num_of_rating', 'origin', 'budget', 'worldwide_gross', 'cast']
credits = ['director', 'writer', 'producer', 'composer', 'cinematographer', 'editor']
data = pd.DataFrame(columns = details + credits)

def get_movie_data(movie):
    index = len(data)
    overview_link = movie.h3.a['href']
    overview_html = requests_session.get(f'https://www.imdb.com/{overview_link[:-16]}', headers=headers).text
    overview_soup = BeautifulSoup(overview_html, 'lxml')
    credits_html = requests_session.get(f'https://www.imdb.com/{overview_link[:-16]}/fullcredits', headers=headers).text
    credits_soup = BeautifulSoup(credits_html, 'lxml')

    title = ' '.join(credits_soup.find('h3').text.split())
    movie_data = [title]

    #  movie details
    detail = {
        'runtime': movie.select_one('span.runtime'),
        'certificate': movie.select_one('span.certificate'),
        'rating': overview_soup.select_one('span.jGRxWM'),
        'num_of_rating': overview_soup.select_one('div.dPVcnq'),
        'origin': overview_soup.select_one('li[data-testid="title-details-origin"] div'),
        'budget': overview_soup.select_one('li[data-testid="title-boxoffice-budget"] div'),
        'worldwide_gross': overview_soup.select_one('li[data-testid="title-boxoffice-cumulativeworldwidegross"] div')
        }
    movie_data += [v.text if v else None for v in detail.values()]

    casts = credits_soup.select_one('table.cast_list')

    if casts:
        movie_data.append(','.join([cast.text.strip() for cast in casts.find_all('td', attrs={'class': None}) if cast.text.strip()]))
    else:
        movie_data.append(None)

    #  movie credits
    for credit in credits:
        c = credits_soup.find('h4', id=credit)
        if c:
            movie_data.append(','.join([cred.text.strip() for cred in c.find_next_sibling().select("a") if cred.text != ""]))
        else:
            movie_data.append(None)

    #  add movie_data into dataframe
    data.loc[index] = movie_data
    print(f'Scraped movie: {index + 1}', end='\r', flush=True)

In [6]:
i = 0
while True:
    driver.get(f'https://www.imdb.com{link}')
    time.sleep(3)

    plain_text = driver.page_source
    soup = BeautifulSoup(plain_text, 'lxml')

    movies = soup.select('div.lister-item-content')
    for movie in movies:
        t = threading.Thread(target=get_movie_data, args=(movie,))
        i += 1
        print(f'start scraping on movie #{i}', end='\r', flush=True)
        time.sleep(random.random() + 2)
        t.start()
    next_button = soup.select_one('a.next-page')
    if next_button:
        link = next_button['href']
    else:
        break

In [4]:
data

,title,runtime,certificate,rating,num_of_rating,origin,budget,worldwide_gross,cast,director,writer,producer,composer,cinematographer,editor


In [147]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 893 entries, 0 to 892
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title            893 non-null    object
 1   runtime          891 non-null    object
 2   certificate      889 non-null    object
 3   rating           893 non-null    object
 4   num_of_rating    893 non-null    object
 5   origin           893 non-null    object
 6   budget           704 non-null    object
 7   worldwide_gross  750 non-null    object
 8   cast             893 non-null    object
 9   director         893 non-null    object
 10  writer           893 non-null    object
 11  producer         893 non-null    object
 12  composer         858 non-null    object
 13  cinematographer  863 non-null    object
 14  editor           890 non-null    object
dtypes: object(15)
memory usage: 143.9+ KB


In [108]:
import threading

test_list = []

def print_index(i):
    test_list.append(i)
    print(i)

for i in range(10):
    t = threading.Thread(target=print_index, args=(i,))
    t.start()

print(test_list)

0
1
2
3
4
5
6
7
8
9
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
